<a href="https://colab.research.google.com/github/grfgrf/exercicios/blob/main/pyspark-%20txtMapReduce%20-%20GoogleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
#***IFs apenas para o notebook não executar novamente em caso de run all cells.

#verifica se openjdk está instalado
if (dpkg -l | grep -qw openjdk-8-jdk-headless) then 
  echo "Ja instalado - openjdk-8-jdk-headless" 
else 
  apt-get install openjdk-8-jdk-headless -qq > /dev/null | echo "openjdk-8-jdk-headless - instalado com sucesso" 
fi

openjdk-8-jdk-headless - instalado com sucesso


In [2]:
%%bash
#download spark-hadoop
if [ -f "spark-3.2.0-bin-hadoop3.2.tgz" ]; then
  echo "Ja baixado - spark-3.2.0-bin-hadoop3.2.tgz"
else 
  wget -q  https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz | echo "spark-3.2.0-bin-hadoop3.2.tgz - baixado com sucesso"
fi

spark-3.2.0-bin-hadoop3.2.tgz - baixado com sucesso


In [3]:
%%bash
#extrair spark-hadoop
if [ -d "spark-3.2.0-bin-hadoop3.2" ]; then
  echo "Ja descompactado - Pasta spark-3.2.0-bin-hadoop3.2"
else 
  tar xf spark-3.2.0-bin-hadoop3.2.tgz | echo "spark-3.2.0-bin-hadoop3.2.tgz - descompactado com sucesso"
fi

spark-3.2.0-bin-hadoop3.2.tgz - descompactado com sucesso


In [4]:
%%bash
#download sherlock.txt
if [ -f "/content/sample_data/sherlock.txt" ]; then
  echo "Ja baixado - sherlock.txt"
else 
  wget -q -O /content/sample_data/sherlock.txt https://www.gutenberg.org/files/1661/1661-0.txt | echo "sherlock.txt - baixado com sucesso"
fi

sherlock.txt - baixado com sucesso


In [5]:
#usa folder do spark como lib pyspark
try:
    findspark
except NameError:
    !pip install -q findspark
    import findspark
    findspark.init('spark-3.2.0-bin-hadoop3.2')
    print("findspark - instalado com sucesso ")
else:
    print("já instalado - findspark")

findspark - instalado com sucesso 


In [6]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [7]:
#cria session e lê sherlock como .rdd
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

#dataframe
txtDF = spark.read.format('text').load('/content/sample_data/sherlock.txt')

#RDD
txtRDD= spark.read.format('text').load('/content/sample_data/sherlock.txt').rdd

#Texto inteiro em RDD com 1 row [k,v]
rddSherlock = spark.sparkContext.wholeTextFiles('/content/sample_data/sherlock.txt',use_unicode=True)


In [8]:
txtDF.head(2)

[Row(value='The Project Gutenberg eBook of The Adventures of Sherlock Holmes, by Arthur Conan Doyle'),
 Row(value='')]

In [9]:
txtRDD.take(2)

[Row(value='The Project Gutenberg eBook of The Adventures of Sherlock Holmes, by Arthur Conan Doyle'),
 Row(value='')]

In [10]:
#Mapeia só [\W]+ lower case
import re

mapSherlock = rddSherlock.map(lambda x : re.sub(r"[\W]+"," ",x[1].lower()))
#mapSherlock.take(1)

In [11]:
#Split por espaço em branco
palavrasSherlock = mapSherlock.flatMap(lambda x: x.split(" "))
palavrasSherlock.take(5)

['', 'the', 'project', 'gutenberg', 'ebook']

In [12]:
#Adiciona coluna com valor 1 para reduce
formatadoSherlock = palavrasSherlock.map(lambda z: (z,1)) 
formatadoSherlock.take(5)

[('', 1), ('the', 1), ('project', 1), ('gutenberg', 1), ('ebook', 1)]

In [13]:
# RDD final com ReduceByKey (reduz keys iguais e soma values)
finalSherlock = formatadoSherlock.reduceByKey(lambda a,b: a+b)
finalSherlock.take(5)

[('', 2), ('the', 5815), ('project', 89), ('gutenberg', 98), ('ebook', 13)]

In [14]:
#lista TOP 10 sorted
#contagem final depende da regra para definir "O que é uma palavra?"
finalSherlock.sortBy(lambda x: x[1],ascending=False).collect()[0:10]

[('the', 5815),
 ('and', 3085),
 ('i', 3038),
 ('to', 2826),
 ('of', 2781),
 ('a', 2700),
 ('in', 1826),
 ('that', 1767),
 ('it', 1749),
 ('you', 1577)]

In [15]:
#Exemplo que tem no Spark Documentation - DataFrame
from pyspark.sql.functions import *
wordCounts = txtDF.select(explode(split(txtDF.value, "\s+")).alias("word")).groupBy("word").count()
wordCounts.orderBy(['count'],ascending=False).show(10)

+----+-----+
|word|count|
+----+-----+
| the| 5412|
|    | 2799|
| and| 2794|
|  of| 2724|
|  to| 2702|
|   a| 2575|
|   I| 2533|
|  in| 1706|
|that| 1557|
| was| 1361|
+----+-----+
only showing top 10 rows

